##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFRecord 및 tf.Example

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/tfrecord"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/tfrecord.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/tfrecord.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/tfrecord.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

데이터를 효율적으로 읽으려면 데이터를 직렬화하여 각각을 선형적으로 읽을 수 있는 파일 세트(각각 100~200MB)에 저장하면 도움이 될 수 있습니다. 데이터가 네트워크를 통해 스트리밍되는 경우에는 특히 그렇습니다. 또한 데이터 전처리를 캐싱하는 데도 유용할 수 있습니다.

TFRecord 형식은 일련의 이진 레코드를 저장하기 위한 단순한 형식입니다.

[프로토콜 버퍼](https://developers.google.com/protocol-buffers/)는 구조화된 데이터의 효율적인 직렬화를 위한 플랫폼 및 언어 간 라이브러리입니다.

프로토콜 메시지는 `.proto` 파일로 정의되며 메시지 유형을 이해하는 가장 쉬운 방법인 경우가 많습니다.

이 노트북은 `tf.Example` 메시지를 생성 및 구문 분석하고 사용한 다음 `tf.Example` 메시지를 `.tfrecord` 파일 사이에서 직렬화하고 쓰기 및 읽기를 수행하는 방법을 보여줍니다.

참고: 유용하기는 하지만 이러한 구조는 선택적입니다. [`tf.data`](https://www.tensorflow.org/guide/data)를 사용하면서 데이터 읽기가 훈련에 계속 병목 현상을 일으키는 경우가 아니라면 TFRecords를 사용하도록 기존 코드를 변환할 필요는 없습니다. 데이터세트 성능에 대한 유용한 정보는 [데이터 입력 파이프라인 성능](https://www.tensorflow.org/guide/performance/datasets)을 참조하세요.

참고: 일반적으로, I/O를 병렬화할 수 있도록 여러 파일에 데이터를 분할해야 합니다(단일 호스트 내에서, 또는 여러 호스트에서). 경험상 데이터를 읽는 호스트 수보다 파일 수가 최소 10배 이상 되어야 합니다. 이와 동시에 I/O 프리페치의 이점을 누릴 수 있으려면 각 파일이 충분히 커야 합니다(최소 10MB 이상, 이상적으로는 100MB 이상). 예를 들어, `X` GB의 데이터가 있고 최대 `N`개의 호스트에서 훈련할 계획이라고 가정해 보겠습니다. 이상적으로, ~ `X/(10*N)`가 10MB 이상(이상적으로는 100MB 이상)이면 데이터를 ~ `10*N` 파일로 분할해야 합니다. 이보다 적으면 병렬 처리의 이점과 I/O 프리페치의 이점을 절충하기 위해 더 적은 수의 샤드를 만들어야 할 수 있습니다.

## !pip install -U tf-hub-nightly<br>import tensorflow_hub as hub<br><br>from tensorflow.keras import layers

In [ ]:
import tensorflow as tf

import numpy as np
import IPython.display as display

## `tf.train.Example`

### `tf.Example`의 데이터 형식

기본적으로 `tf.Example`은 `{"string": tf.train.Feature}` 매핑입니다.

`tf.train.Feature` 메시지 유형은 다음 3가지 유형 중 하나를 허용할 수 있습니다([`.proto` 파일](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/feature.proto) 참조). 다른 대부분의 일반 형식은 다음 중 하나로 강제 변환할 수 있습니다.

1. `tf.train.BytesList`(다음 유형을 강제 변환할 수 있음)

- `string`
- `byte`

1. `tf.train.FloatList`(다음 유형을 강제 변환할 수 있음)

- `float` ( `float32` )
- `double` ( `float64` )

1. `tf.train.Int64List`(다음 유형을 강제 변환할 수 있음)

- `bool`
- `enum`
- `int32`
- `uint32`
- `int64`
- `uint64`

표준 TensorFlow 유형을 `tf.Example` 호환 `tf.train.Feature` 로 변환하려면 아래 바로 가기 함수를 사용할 수 있습니다. 각 함수는 스칼라 입력 값을 받아들여 위의 세 가지 `list` 유형 중 하나를 포함하는 `tf.train.Feature`를 반환합니다.

In [ ]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

참고: 단순화를 위해 이 예에서는 스칼라 입력만 사용합니다. 스칼라가 아닌 특성을 처리하는 가장 간단한 방법은 `tf.io.serialize_tensor`를 사용하여 텐서를 이진 문자열로 변환하는 것입니다. 문자열은 tensorflow에서 스칼라입니다. 이진 문자열을 다시 텐서로 변환하려면 `tf.io.parse_tensor`를 사용하세요.

다음은 이러한 함수의 작동 방식을 보여주는 몇 가지 예입니다. 다양한 입력 유형과 표준화된 출력 유형에 주목하세요. 함수의 입력 유형이 위에서 언급 한 강제 변환할 수 있는 유형 중 하나와 일치하지 않으면 함수에서 예외가 발생합니다(예: `1.0`은 부동 소수점이어서 `_int64_feature(1.0)` 오류가 발생하므로 대신 `_float_feature` 함수와 함께 사용해야 함).

In [ ]:
print(_bytes_feature(b'test_string'))
print(_bytes_feature(u'test_bytes'.encode('utf-8')))

print(_float_feature(np.exp(1)))

print(_int64_feature(True))
print(_int64_feature(1))

`.SerializeToString` 메서드를 사용하여 모든 proto 메시지를 이진 문자열로 직렬화할 수 있습니다.

In [ ]:
feature = _float_feature(np.exp(1))

feature.SerializeToString()

### `tf.Example` 메시지 작성하기

기존 데이터에서 `tf.Example` 메시지를 작성한다고 가정해 보겠습니다. 실제로 데이터세트는 어디서든 올 수 있지만 단일 관측에서 `tf.Example` 메시지를 작성하는 절차는 같습니다.

1. 각 관측 내에서 각 값은 위의 함수 중 하나를 사용하여 3가지 호환 유형 중 하나를 포함하는 `tf.train.Feature`로 변환해야 합니다.

2. 특성 이름 문자열을 #1에서 생성된 인코딩된 특성 값과 연결하는 맵(사전)을 생성합니다.

3. 2단계에서 생성한 맵은 [`Features` 메시지](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/feature.proto#L85)로 변환됩니다.

이 노트북에서는 NumPy를 사용하여 데이터세트를 만듭니다.

이 데이터세트에는 4가지 특성이 있습니다.

- 확률이 동일한 부울 특성 `False` 또는 `True`
- `[0, 5]`에서 균일하게 무작위로 선택된 정수 특성
- 정수 특성을 인덱스로 사용하여 문자열 테이블에서 생성된 문자열 특성
- 표준 정규 분포의 부동 특성

위의 각 분포에서 독립적으로 동일하게 분포된 관측 값 10,000개로 구성된 표본을 고려하겠습니다.

In [ ]:
# The number of observations in the dataset.
n_observations = int(1e4)

# Boolean feature, encoded as False or True.
feature0 = np.random.choice([False, True], n_observations)

# Integer feature, random from 0 to 4.
feature1 = np.random.randint(0, 5, n_observations)

# String feature.
strings = np.array([b'cat', b'dog', b'chicken', b'horse', b'goat'])
feature2 = strings[feature1]

# Float feature, from a standard normal distribution.
feature3 = np.random.randn(n_observations)

이러한 각 특성은 `_bytes_feature` , `_float_feature` , `_int64_feature` 중 하나를 사용하여 `tf.Example` 호환 유형으로 강제 변환할 수 있습니다. 그런 다음, 이러한 인코딩된 특성에서 `tf.Example` 메시지를 작성할 수 있습니다.

In [ ]:
def serialize_example(feature0, feature1, feature2, feature3):
  """
  Creates a tf.train.Example message ready to be written to a file.
  """
  # Create a dictionary mapping the feature name to the tf.train.Example-compatible
  # data type.
  feature = {
      'feature0': _int64_feature(feature0),
      'feature1': _int64_feature(feature1),
      'feature2': _bytes_feature(feature2),
      'feature3': _float_feature(feature3),
  }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

예를 들어, 데이터세트에서 얻은 단일 관측 `[False, 4, bytes('goat'), 0.9876]`이 있다고 하겠습니다. `create_message()`를 사용하여 이 관측에 대한 `tf.Example` 메시지를 작성하고 인쇄할 수 있습니다. 각 단일 관측은 위의 조건에 따라 `Features` 메시지로 작성됩니다. `tf.Example` [메시지](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/example.proto#L88)는 `Features` 메시지 주변에서 단지 래퍼로만 작동합니다.

In [ ]:
# This is an example observation from the dataset.

example_observation = []

serialized_example = serialize_example(False, 4, b'goat', 0.9876)
serialized_example

메시지를 디코딩하려면 `tf.train.Example.FromString` 메서드를 사용합니다.

In [ ]:
example_proto = tf.train.Example.FromString(serialized_example)
example_proto

## TF 레코드 형식 세부 사항

TFRecord 파일에는 일련의 레코드가 포함됩니다. 파일은 순차적으로만 읽을 수 있습니다.

각 레코드에는 데이터 페이로드에 대한 바이트 문자열과 데이터 길이 및 무결성 검사를 위한 CRC32C(Castagnoli 다항식을 사용하는 32비트 CRC) 해시가 포함됩니다.

각 레코드는 다음 형식으로 저장됩니다.

```
uint64 length
uint32 masked_crc32_of_length
byte   data[length]
uint32 masked_crc32_of_data
```

이 레코드는 함께 연결되어 파일을 생성합니다. CRC는 [여기서 설명](https://en.wikipedia.org/wiki/Cyclic_redundancy_check)하며 CRC 마스크는 다음과 같습니다.

```
masked_crc = ((crc >> 15) | (crc << 17)) + 0xa282ead8ul
```


참고: TFRecord 파일에서 `tf.train.Example`을 사용할 필요는 없습니다. `tf.train.Example`은 사전을 바이트 문자열로 직렬화하는 방법일 뿐입니다. TensorFlow에서 디코딩할 수 있는 모든 바이트 문자열은 TFRecord 파일에 저장할 수 있습니다. 텍스트 줄, JSON(`tf.io.decode_json_example` 사용), 인코딩된 이미지 데이터 또는 직렬화된 `tf.Tensors`(`tf.io.serialize_tensor`/`tf.io.parse_tensor` 사용) 등을 예로 들 수 있습니다. 더 많은 옵션은 `tf.io` 모듈을 참조하세요.

## `tf.data`를 사용한 TFRecord 파일

`tf.data` 모듈은 TensorFlow에서 데이터를 읽고 쓰기 위한 도구를 제공합니다.

### TFRecord 파일 작성하기

데이터를 데이터세트로 가져오는 가장 쉬운 방법은 `from_tensor_slices` 메서드를 사용하는 것입니다.

배열에 적용했을 때는 스칼라의 데이터세트를 반환합니다.

In [ ]:
tf.data.Dataset.from_tensor_slices(feature1)

배열의 튜플에 적용하면 튜플의 데이터세트를 반환합니다.

In [ ]:
features_dataset = tf.data.Dataset.from_tensor_slices((feature0, feature1, feature2, feature3))
features_dataset

In [ ]:
# Use `take(1)` to only pull one example from the dataset.
for f0,f1,f2,f3 in features_dataset.take(1):
  print(f0)
  print(f1)
  print(f2)
  print(f3)

`tf.data.Dataset.map` 메서드를 사용하여 `Dataset` 각 요소에 함수를 적용합니다.

매핑된 함수는 TensorFlow 그래프 모드에서 작동해야 합니다(`tf.Tensors`에서 작동하고 이를 반환해야 함). `serialize_example`과 같이 텐서가 아닌 함수는 `tf.py_function`으로 래핑하여 호환되도록 만들 수 있습니다.

`tf.py_function` 사용하려면 다른 방식으로는 사용할 수 없는 형상과 유형 정보를 지정해야 합니다.

In [ ]:
def tf_serialize_example(f0,f1,f2,f3):
  tf_string = tf.py_function(
    serialize_example,
    (f0, f1, f2, f3),  # Pass these args to the above function.
    tf.string)      # The return type is `tf.string`.
  return tf.reshape(tf_string, ()) # The result is a scalar.

In [ ]:
tf_serialize_example(f0, f1, f2, f3)

이 함수를 데이터세트의 각 요소에 적용합니다.

In [ ]:
serialized_features_dataset = features_dataset.map(tf_serialize_example)
serialized_features_dataset

In [ ]:
def generator():
  for features in features_dataset:
    yield serialize_example(*features)

In [ ]:
serialized_features_dataset = tf.data.Dataset.from_generator(
    generator, output_types=tf.string, output_shapes=())

In [ ]:
serialized_features_dataset

그리고 TFRecord 파일에 이 내용을 작성합니다.

In [ ]:
filename = 'test.tfrecord'
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(serialized_features_dataset)

### TFRecord 파일 읽기

`tf.data.TFRecordDataset` 클래스를 사용하여 TFRecord 파일을 읽을 수도 있습니다.

`tf.data`를 사용하여 TFRecord 파일을 소비하기 위한 자세한 내용은 [여기](https://www.tensorflow.org/guide/datasets#consuming_tfrecord_data)에서 확인할 수 있습니다.

`TFRecordDataset`를 사용하면 입력 데이터를 표준화하고 성능을 최적화하는 데 유용할 수 있습니다.

In [ ]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

이 시점에서 데이터세트에는 직렬화된 `tf.train.Example` 메시지가 포함됩니다. 반복 실행하면 메시지가 스칼라 문자열 텐서로 반환됩니다.

`.take` 메서드를 사용하여 처음 10개의 레코드만 표시합니다.

참고: `tf.data.Dataset`의 반복 실행은 즉시 실행이 활성화된 경우에만 작동합니다.

In [ ]:
for raw_record in raw_dataset.take(10):
  print(repr(raw_record))

이러한 텐서는 아래 함수를 이용해 구문 분석할 수 있습니다. 여기서 `feature_description`이 필요한 이유는 데이터세트가 그래프 실행을 사용하고 형상과 유형을 빌드하기 위해 이 설명을 필요로 하기 때문입니다.

In [ ]:
# Create a description of the features.
feature_description = {
    'feature0': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'feature1': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'feature2': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'feature3': tf.io.FixedLenFeature([], tf.float32, default_value=0.0),
}

def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

또는, `tf.parse example`를 사용하여 전체 배치를 한 번에 구문 분석합니다. `tf.data.Dataset.map` 메서드를 사용하여 데이터세트의 각 항목에 이 함수를 적용합니다.

In [ ]:
parsed_dataset = raw_dataset.map(_parse_function)
parsed_dataset

즉시 실행을 사용하여 데이터세트에 관측 값을 표시합니다. 이 데이터세트에는 10,000개의 관측 값이 있지만 처음 10개만 표시됩니다. 데이터는 특성의 사전으로 표시됩니다. 각 항목은 `tf.Tensor`이며 이 텐서의 `numpy` 요소는 특성 값을 표시합니다.

In [ ]:
for parsed_record in parsed_dataset.take(10):
  print(repr(parsed_record))

여기서 `tf.parse_example` 함수는 `tf.Example` 필드를 표준 텐서로 풀어 넣습니다.

## Python에서 TFRecord 파일

`tf.io` 모듈에는 TFRecord 파일을 읽고 쓰기 위한 순수 Python 함수도 포함되어 있습니다.

### TFRecord 파일 작성하기

다음으로, `test.tfrecord` 파일에 10,000개의 관측 값을 작성합니다. 각 관측 값은 `tf.Example` 메시지로 변환된 다음 파일에 작성됩니다. 그러면 `test.tfrecord` 파일이 생성되었는지 확인할 수 있습니다.

In [ ]:
# Write the `tf.train.Example` observations to the file.
with tf.io.TFRecordWriter(filename) as writer:
  for i in range(n_observations):
    example = serialize_example(feature0[i], feature1[i], feature2[i], feature3[i])
    writer.write(example)

In [ ]:
!du -sh {filename}

### TFRecord 파일 읽기

이 직렬화된 텐서는 `tf.train.Example.ParseFromString`을 사용하여 쉽게 구문 분석할 수 있습니다.

In [ ]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

In [ ]:
for raw_record in raw_dataset.take(1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

That returns a `tf.train.Example` proto which is dificult to use as is, but it's fundamentally a representation of a:

```
Dict[str,
     Union[List[float],
           List[int],
           List[str]]]
```

The following code manually converts the `Example` to a dictionary of NumPy arrays, without using TensorFlow Ops. Refer to [the PROTO file](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/feature.proto) for detials.

In [ ]:
result = {}
# example.features.feature is the dictionary
for key, feature in example.features.feature.items():
  # The values are the Feature objects which contain a `kind` which contains:
  # one of three fields: bytes_list, float_list, int64_list

  kind = feature.WhichOneof('kind')
  result[key] = np.array(getattr(feature, kind).value)

result

## 연습: 이미지 데이터 읽기 및 쓰기

이번 연습은 TFRecord를 사용하여 이미지 데이터를 읽고 쓰는 방법을 보여주는 엔드 투 엔드 예제입니다. 이미지를 입력 데이터로 사용하여 데이터를 TFRecord 파일로 작성한 다음 파일을 다시 읽고 이미지를 표시합니다.

예를 들어, 이 방법은 같은 입력 데이터세트에서 여러 모델을 사용하려는 경우에 유용합니다. 이미지 데이터를 그대로 저장하는 대신 TFRecord 형식으로 사전 처리할 수 있으며 이후의 모든 처리 및 모델링에 사용할 수 있습니다.

먼저, 눈 속 고양이를 보여주는 [이 이미지](https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg)와 건설 중인 NYC의 Williamsburg Bridge를 보여주는 [이 사진](https://upload.wikimedia.org/wikipedia/commons/f/fe/New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg)을 다운로드하겠습니다.

### 이미지 가져오기

In [ ]:
cat_in_snow  = tf.keras.utils.get_file(
    '320px-Felis_catus-cat_on_snow.jpg',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/320px-Felis_catus-cat_on_snow.jpg')

williamsburg_bridge = tf.keras.utils.get_file(
    '194px-New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/194px-New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg')

In [ ]:
display.display(display.Image(filename=cat_in_snow))
display.display(display.HTML('Image cc-by: <a "href=https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg">Von.grzanka</a>'))

In [ ]:
display.display(display.Image(filename=williamsburg_bridge))
display.display(display.HTML('<a "href=https://commons.wikimedia.org/wiki/File:New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg">From Wikimedia</a>'))

### TFRecord 파일 작성하기

이전과 마찬가지로 특성을 `tf.Example`과 호환되는 유형으로 인코딩합니다. 그러면 원시 이미지 문자열 특성과 높이, 너비, 깊이 및 임의의 `label` 특성이 저장됩니다.  후자는 고양이 이미지와 다리 이미지를 구별하는 파일을 작성할 때 사용됩니다. 고양이 이미지에는 `0`을 사용하고 다리 이미지에는 `1`을 사용합니다.

In [ ]:
image_labels = {
    cat_in_snow : 0,
    williamsburg_bridge : 1,
}

In [ ]:
# This is an example, just using the cat image.
image_string = open(cat_in_snow, 'rb').read()

label = image_labels[cat_in_snow]

# Create a dictionary with features that may be relevant.
def image_example(image_string, label):
  image_shape = tf.io.decode_jpeg(image_string).shape

  feature = {
      'height': _int64_feature(image_shape[0]),
      'width': _int64_feature(image_shape[1]),
      'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }

  return tf.train.Example(features=tf.train.Features(feature=feature))

for line in str(image_example(image_string, label)).split('\n')[:15]:
  print(line)
print('...')

이제 모든 특성이 `tf.Example` 메시지에 저장됩니다. 그 다음, 위의 코드를 함수화하고 예제 메시지를 `images.tfrecords` 이름의 파일에 작성합니다.

In [ ]:
# Write the raw image files to `images.tfrecords`.
# First, process the two images into `tf.train.Example` messages.
# Then, write to a `.tfrecords` file.
record_file = 'images.tfrecords'
with tf.io.TFRecordWriter(record_file) as writer:
  for filename, label in image_labels.items():
    image_string = open(filename, 'rb').read()
    tf_example = image_example(image_string, label)
    writer.write(tf_example.SerializeToString())

In [ ]:
!du -sh {record_file}

### TFRecord 파일 읽기

이제 `images.tfrecords` 파일이 얻었으며 여기에 있는 레코드를 반복 실행하여 작성한 내용을 다시 읽을 수 있습니다. 이 예에서는 이미지만 생성하기 때문에 유일하게 필요한 특성은 원시 이미지 문자열입니다. 위에서 설명한 getter, 즉 `example.features.feature['image_raw'].bytes_list.value[0]`를 이용해 이를 추출합니다. 또한 레이블을 이용해 어떤 레코드가 고양이이고 어떤 것이 다리인지 결정할 수 있습니다.

In [ ]:
raw_image_dataset = tf.data.TFRecordDataset('images.tfrecords')

# Create a dictionary describing the features.
image_feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

TFRecord 파일에서 이미지를 복구합니다.

In [ ]:
for image_features in parsed_image_dataset:
  image_raw = image_features['image_raw'].numpy()
  display.display(display.Image(data=image_raw))